# Inicialização

In [ ]:
URL="https://pokeapi.co/api/v2/"
BUSCA="pokemon-species/"
BUSCA2="pokemon/"
BUSCA4="type/"
BUSCA5="move/"
ALCANCE=151
import requests
import json
import math
import random
ALIADO="Digite o nome do seu pokemon: "
INIMIGO="Digite o nome do pokemon inimigo: "
TESTE=100000
FUGA=0.7

#peso->kg
#altura->cm

# FUNÇÕES

In [ ]:
def faz_dex(pokemon_dic,pokemon_dic2,cont):
  lista_egg_group=[]
  lista_tipos=[]
  dicio={}
  if(pokemon_dic[cont]["evolves_from_species"]==None):
    pre_evo=None
  else:
    pre_evo=(pokemon_dic[cont]["evolves_from_species"]["name"])
  for i in range(len(pokemon_dic[cont]["egg_groups"])):
    lista_egg_group.append(pokemon_dic[cont]["egg_groups"][i]["name"])
  for i in range(len(pokemon_dic2[cont]["types"])):
    lista_tipos.append(pokemon_dic2[cont]["types"][i]["type"]["name"])
  dicio={"Nome":pokemon_dic[cont]["name"].upper(),"tipos":lista_tipos,"Genus":pokemon_dic[cont]["genera"][7]["genus"],"Cor":pokemon_dic[cont]["color"]["name"],"Texto":pokemon_dic[cont]["flavor_text_entries"][1]["flavor_text"],"pre-evolução":pre_evo,"habitat":pokemon_dic[cont]["habitat"]["name"],"lendario":pokemon_dic[cont]["is_legendary"],"forma":pokemon_dic[cont]["shape"]["name"],"Grupo de Ovos":lista_egg_group,"Altura":pokemon_dic2[cont]["height"]*10,"Peso":pokemon_dic2[cont]["weight"]*0.1,"taxa de captura":pokemon_dic[cont]["capture_rate"],"atributos":{"Vida":pokemon_dic2[cont]["stats"][0]["base_stat"],
"Ataque":pokemon_dic2[cont]["stats"][1]["base_stat"],
"Defesa":pokemon_dic2[cont]["stats"][2]["base_stat"],
"Ataque especial":pokemon_dic2[cont]["stats"][3]["base_stat"],
"Defesa especial":pokemon_dic2[cont]["stats"][4]["base_stat"],
"Velocidade":pokemon_dic2[cont]["stats"][5]["base_stat"]}}    
  return dicio

In [ ]:
def grava_json(arquivo,link="teste.json"):
  with open(link,"a") as f:
    json.dump(arquivo,f, indent=4)

In [ ]:
def requesta_a_festa(link):
  entrada=requests.get(link).json()
  return entrada

In [ ]:
def busca_poke(pokemon,poke_dex):
  for i in poke_dex:
    if(poke_dex[i].get("Nome")==pokemon):
      return i
  print("Pokemon invalido")
  return -1

In [ ]:
def cria_poke(MSG):
  id=-1
  while(id<0):
    pokemon=input(MSG).upper()
    id=busca_poke(pokemon,poke_dex)
  pokemon=poke_dex[id]
  return pokemon
# print(pokemon)

In [ ]:
def busca_atq(atq,pokemon):
  dicio={}
  dicio=requesta_a_festa(URL+BUSCA5+atq)
  for i in dicio["learned_by_pokemon"]:
    if(i["name"].upper()==pokemon["Nome"]):
      return dicio
  print("Ataque invalido")
  return None 

In [ ]:
def cria_atq(pokemon):
  atq=None
  while(atq==None):
    atq=input("digite o nome do seu ataque: ").lower()
    atq=atq.replace(" ","-")
    atq=busca_atq(atq,pokemon)
  # print(atq)
  return atq

In [ ]:
def faz_status_OD(atq,pokemon_a,pokemon_e):
  #0=OFENSIVO
  #1=DEFENSIVO
  lista_OD=[0,0]
  if(atq["damage_class"]["name"]=="special"):
    lista_OD[0]=pokemon_a["atributos"]["Ataque especial"]
    lista_OD[1]=pokemon_e["atributos"]["Defesa especial"]
  else:
    lista_OD[0]=pokemon_a["atributos"]["Ataque"]
    lista_OD[1]=pokemon_e["atributos"]["Defesa"]
  return lista_OD
# print(status_ofensivo)
# print(statuds_defensivo)

In [ ]:
def faz_stab(atq,pokemon):
  if(atq["type"]["name"] in pokemon["tipos"]):
    stab=1.5
  else:
    stab=1
  return stab
# print(stab)

In [ ]:
def cria_dicio_res():
  dicio_res={}
  for i in range(18):
    dicio=requesta_a_festa(URL+BUSCA4+str(i+1))
    dicio_res[dicio["name"]]=dicio
    
  return dicio_res

In [ ]:
def faz_resistencia(pokemon,atq,dicio_res):
  resistencia_multi=1
  for i in pokemon["tipos"]:
    for k in dicio_res[i]["damage_relations"]["double_damage_from"]:
      if atq["type"]["name"]==k["name"]:
        resistencia_multi*=2
    for k in dicio_res[i]["damage_relations"]["half_damage_from"]:
      if atq["type"]["name"]== k["name"]:
        resistencia_multi=resistencia_multi/2
    for k in dicio_res[i]["damage_relations"]["no_damage_from"]:
        if atq["type"]["name"]==k["name"]:
          resistencia_multi=0
  return(resistencia_multi) 

In [ ]:
def total_vida(pokemon,EV=252,IV=31,LVL=100):
  vida=math.floor(0.01*(2*pokemon["atributos"]["Vida"]+IV+math.floor(0.25*EV))*LVL)+LVL+10
  return vida

In [ ]:
def total_stats(stat,EV=252,IV=31,LVL=100,NATURE=1):
  stat=math.floor(0.01*((2*stat+IV+math.floor(0.25*EV))*LVL)+5)*NATURE
  return stat

In [ ]:
def faz_atq(pokemon_a,pokemon_e,atq,poder_atq,status_ofensivo,status_defensivo,stab,resistencia_multi):
    aleatorio=1
    if(poder_atq==None):
      return 0
    if(resistencia_multi==0):
      return 0
    else:
      dano= round(((((42)* poder_atq * (status_ofensivo/status_defensivo))/50)+2)*stab*resistencia_multi*aleatorio,0)
      if(atq["accuracy"]==None):
        multi_mira=1
      else:
        multi_mira=atq["accuracy"]/100
      dano*=multi_mira
    return dano


In [ ]:
def faz_relogio(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,pokemon_e):
  maior=0
  for i in pokemon_atributos_a["atq"]:
    atq=pokemon_atributos_a["atq"][i]
    stab=faz_stab(atq,pokemon_a)
    poder_atq=atq["power"]
    lista_OD=faz_status_OD(atq,pokemon_a,pokemon_e)
    if(atq["damage_class"]["name"]=="special"):
      EV_a=pokemon_atributos_a["EVs"]["Ataque especial"]
      IV_a=pokemon_atributos_a["IVs"]["Ataque especial"]
      EV_e=pokemon_atributos_e["EVs"]["Defesa especial"]
      IV_e=pokemon_atributos_e["IVs"]["Defesa especial"]
    else:
      EV_a=pokemon_atributos_a["EVs"]["Ataque"]
      IV_a=pokemon_atributos_a["IVs"]["Ataque"]
      EV_e=pokemon_atributos_e["EVs"]["Defesa"]
      IV_e=pokemon_atributos_e["IVs"]["Defesa"]
    status_ofensivo=total_stats(lista_OD[0],EV_a,IV_a)
    status_defensivo=total_stats(lista_OD[1],EV_e,IV_e)
    resistencia_multi=faz_resistencia(pokemon_e,atq,dicio_res)
    dano=faz_atq(pokemon_a,pokemon_e,atq,poder_atq,status_ofensivo,status_defensivo,stab,resistencia_multi)
    if(dano>maior):
      maior_atq=atq
      maior=dano
      # print(atq["name"])
  return maior_atq


In [ ]:
def cria_pra_batalha(pokemon_dic, MSG):
  pokemon=cria_poke(MSG)
  for i in pokemon_dic:
    if(pokemon_dic[i]["name"].upper()==pokemon["Nome"]):
      id=i
      break
  pokemon_atributos_a={}
  pokemon_atributos_a[id]={"name":"teste","IVs":{"Vida":0,"Ataque":0,"Defesa":0,"Ataque especial":0,"Defesa especial":0,"Velocidade":0},
                                      "EVs":{"Vida":0,"Ataque":0,"Defesa":0,"Ataque especial":0,"Defesa especial":0,"Velocidade":0},
                                      "atq":{1:{"name":"","type":{"name":""},"damage_class":{"name":""},"power":0,"accuracy":0},2:{"name":"","type":{"name":""},"damage_class":{"name":""},"power":0,"accuracy":0},3:{"name":"","type":{"name":""},"damage_class":{"name":""},"power":0,"accuracy":0},4:{"name":"","type":{"name":""},"damage_class":{"name":""},"power":0,"accuracy":0}}}                                    


  pokemon_atributos_a[id]["name"]=pokemon["Nome"]
  for i in pokemon_atributos_a[id]["IVs"]:
    IV=0
    while IV<=0 or IV>31:
      print("digite seu IV de ",i)
      IV=int(input())
    pokemon_atributos_a[id]["IVs"][i]=IV
  soma_EV=0
  while soma_EV>510 or soma_EV<=0:
    for i in pokemon_atributos_a[id]["EVs"]:
      EV=-1
      while EV<0 or EV>255:
        print("digite seu EV de ",i)
        EV=int(input())
      soma_EV+=EV
      if(soma_EV>510):
        print("Soma de EV excede 510.")
        soma_EV=0
        break
      pokemon_atributos_a[id]["EVs"][i]=EV
      
  for i in range(4):
    atq= cria_atq(pokemon)
    pokemon_atributos_a[id]["atq"][i+1]={"name":atq["name"],"damage_class":{"name":atq["damage_class"]["name"]},"type":{"name":atq["type"]["name"]},"power":atq["power"],"accuracy":atq["accuracy"]}
  return pokemon_atributos_a



In [ ]:
def pega_id(pokemon):
  for i in pokemon_dic:
    if(pokemon_dic[i]["name"].upper()==pokemon["Nome"]):
      id=i
      return i

In [ ]:
def COMBATE(pokemon_a,pokemon_e,atq_a,atq_e,stab_a,stab_e,resistencia_multi_a,resistencia_multi_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,maior_atq_a,maior_atq_e,velocidade_a,velocidade_e):
  vitoria_e=0
  vitoria_a=0
  for i in range(TESTE):
    vida_e=total_vida(pokemon_e)
    vida_a=total_vida(pokemon_a)
    while(vida_a>0 or vida_e>0):
      if(velocidade_a>velocidade_e):
        dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
        vida_e-=dano
        if(vida_e<=0):
          vitoria_a+=1
          break
        dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
        vida_a-=dano
        if(vida_a<=0):
          vitoria_e+=1
          break
      elif(velocidade_e>velocidade_a):
        dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
        vida_a-=dano
        if(vida_a<=0):
          vitoria_e+=1
          break
        dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
        vida_e-=dano
        if(vida_e<=0):
          vitoria_a+=1
          break
      else:
        empate=random.randrange(1,3)
        if(empate==1):
          dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
          vida_e-=dano
          if(vida_e<=0):
            vitoria_a+=1
            break
          dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
          vida_a-=dano
          if(vida_a<=0):
            vitoria_e+=1
            break
        else:
          dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
          vida_a-=dano
          if(vida_a<=0):
            vitoria_e+=1
            break
          dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
          vida_e-=dano
          if(vida_e<=0):
            vitoria_a+=1
            break       
  return vitoria_a

In [ ]:
def troca_ou_fica(pokemon_a,pokemon_atributos_a,pokemon_atributos_e,poke_atual_a,poke_atual_e,vida_time_a,vida_e):
  maior_vit=-1
  velocidade_a=total_stats(poke_atual_a["atributos"]["Velocidade"])
  velocidade_e=total_stats(poke_atual_e["atributos"]["Velocidade"])
  for id_a in pokemon_a:
    print("Testando:",pokemon_a[id_a]["Nome"],"vs",poke_atual_e["Nome"])
    atq_a=faz_relogio(pokemon_atributos_a[str(id_a)],pokemon_atributos_e[str(pega_id(poke_atual_e))],pokemon_a[id_a],poke_atual_e)
    atq_e=faz_relogio(pokemon_atributos_e[str(pega_id(poke_atual_e))],pokemon_atributos_a[str(id_a)],poke_atual_e,pokemon_a[id_a])
    if pokemon_a[id_a]["Nome"]==poke_atual_a["Nome"]:
      atq_atual=atq_a
    lista_OD=faz_status_OD(atq_a,pokemon_a[id_a],poke_atual_e)
    status_ofensivo_a=total_stats(lista_OD[0])
    status_defensivo_e=total_stats(lista_OD[1])
    lista_OD=faz_status_OD(atq_e,poke_atual_e,pokemon_a[id_a])
    status_ofensivo_e=total_stats(lista_OD[0])
    status_defensivo_a=total_stats(lista_OD[1])
    vitoria=chama_COMBATE2(pokemon_a[id_a],poke_atual_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e,vida_time_a[id_a],vida_e)
    if pokemon_a[id_a]["Nome"]==poke_atual_a["Nome"]:
      vitoria_atual=vitoria
    if vitoria/TESTE>maior_vit:
      maior_vit=vitoria/TESTE
      maior_poke_a=pokemon_a[id_a]
      maior_atq_a=atq_a    
  if maior_poke_a["Nome"]!=poke_atual_a["Nome"] and vitoria_atual/TESTE<0.7 and maior_vit>0.5:
    troca=[True,maior_poke_a,maior_atq_a]
  else:
    troca=[False,poke_atual_a,atq_atual]
  print("Troca:",str(troca[0]))
  return troca




In [ ]:
def chama_COMBATE(pokemon_a,pokemon_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e):
  maior_atq_a=atq_a["power"]
  maior_atq_e=atq_e["power"]
  stab_a=faz_stab(atq_a,pokemon_a)
  stab_e=faz_stab(atq_e,pokemon_e)
  resistencia_multi_a=faz_resistencia(pokemon_a,atq_e,dicio_res)
  resistencia_multi_e=faz_resistencia(pokemon_e,atq_a,dicio_res)
  vitoria= COMBATE(pokemon_a,pokemon_e,atq_a,atq_e,stab_a,stab_e,resistencia_multi_a,resistencia_multi_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,maior_atq_a,maior_atq_e,velocidade_a,velocidade_e,)
  return vitoria

In [ ]:
def faz_atq2(atq,poder_atq,status_ofensivo,status_defensivo,stab,resistencia_multi):
  aleatorio=random.randrange(217,256)/255
  mira=atq["accuracy"]
  if(random.randrange(1,101)>mira):
    return 0
  else:
    if(resistencia_multi==0):
      return 0
    else:
      if(random.randrange(1,17)>=16):
        aleatorio=1
        dano= round(((((42)* poder_atq * (status_ofensivo/status_defensivo))/50)+2)*stab*resistencia_multi*aleatorio*2,0)  
      else:
        dano= round(((((42)* poder_atq * (status_ofensivo/status_defensivo))/50)+2)*stab*resistencia_multi*aleatorio,0)
  return dano

In [ ]:
def universo_move(id,pokemon_a,pokemon_e):
  maior=0
  for i in pokemon_dic2[id]["moves"]:
    atq=busca_atq(i["move"]["name"],pokemon_a)
    vida_e=total_vida(pokemon_e)
    stab=faz_stab(atq,pokemon_a)
    poder_atq=atq["power"]
    lista_OD=faz_status_OD(atq,pokemon_a,pokemon_e)
    status_ofensivo=total_stats(lista_OD[0])
    status_defensivo=total_stats(lista_OD[1])
    resistencia_multi=faz_resistencia(pokemon_e,atq,dicio_res)
    dano=faz_atq(pokemon_a,pokemon_e,atq,poder_atq,status_ofensivo,status_defensivo,stab,resistencia_multi)
    if(dano>maior):
      retorna_atq=atq
      maior=dano
  return retorna_atq

In [ ]:
def mini_universo_combate(pokemon_a,pokemon_e,id):
  lista_dano=[0,0]
  lista_atq=[0,0]
  lista_tipo=[0,0]
  lista_final=[]
  for i in pokemon_dic2[id]["moves"]:
    atq=busca_atq(i["move"]["name"],pokemon_a)
    vida_e=total_vida(pokemon_e)
    stab=faz_stab(atq,pokemon_a)
    poder_atq=atq["power"]
    lista_OD=faz_status_OD(atq,pokemon_a,pokemon_e)
    status_ofensivo=total_stats(lista_OD[0])
    status_defensivo=total_stats(lista_OD[1])
    resistencia_multi=faz_resistencia(pokemon_e,atq,dicio_res)
    dano=faz_atq(pokemon_a,pokemon_e,atq,poder_atq,status_ofensivo,status_defensivo,stab,resistencia_multi)
    if(dano>lista_dano[0]):
      lista_dano[0]=dano
      lista_atq[0]=atq["name"]
      lista_tipo[0]=atq["damage_class"]["name"]
      if(atq["accuracy"]==None):
        acc=100
      else:
        acc=atq["accuracy"]
    elif (dano>lista_dano[1] and atq["accuracy"]==100):
      lista_dano[1]=dano
      lista_atq[1]=atq["name"]
      lista_tipo[1]=atq["damage_class"]["name"]
  lista_final.append(lista_dano)
  lista_final.append(lista_atq)
  lista_final.append(lista_tipo)
  lista_final.append(acc)
  return lista_final   


In [ ]:
def abre_json(file):
  with open(file) as f:
    item=json.load(f)
  return item

In [ ]:
def COMBATE2(atq_a,atq_e,stab_a,stab_e,resistencia_multi_a,resistencia_multi_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,maior_atq_a,maior_atq_e,velocidade_a,velocidade_e,vida_a,vida_e):
  vitoria_e=0
  vitoria_a=0
  max_a=vida_a
  max_e=vida_e
  for i in range(TESTE):
    vida_a=max_a
    vida_e=max_e
    while(vida_a>0 or vida_e>0):
      if(velocidade_a>velocidade_e):
        dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
        vida_e-=dano
        if(vida_e<=0):
          vitoria_a+=1
          break
        dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
        vida_a-=dano
        if(vida_a<=0):
          vitoria_e+=1
          break
      elif(velocidade_e>velocidade_a):
        dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
        vida_a-=dano
        if(vida_a<=0):
          vitoria_e+=1
          break
        dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
        vida_e-=dano
        if(vida_e<=0):
          vitoria_a+=1
          break
      else:
        empate=random.randrange(1,3)
        if(empate==1):
          dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
          vida_e-=dano
          if(vida_e<=0):
            vitoria_a+=1
            break
          dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
          vida_a-=dano
          if(vida_a<=0):
            vitoria_e+=1
            break
        else:
          dano=faz_atq2(atq_e,maior_atq_e,status_ofensivo_e,status_defensivo_a,stab_e,resistencia_multi_a)
          vida_a-=dano
          if(vida_a<=0):
            vitoria_e+=1
            break
          dano=faz_atq2(atq_a,maior_atq_a,status_ofensivo_a,status_defensivo_e,stab_a,resistencia_multi_e)
          vida_e-=dano
          if(vida_e<=0):
            vitoria_a+=1
            break
  return vitoria_a

In [ ]:
def escolhe_aleatorio(equipe):
  lista=[]
  maior=-1
  for i in equipe:
    lista.append([i,random.randrange(1,1000)])
  for i in lista:
    if i[1]>maior:
      maior=i[1]
      id=i[0]
  return id



In [ ]:
def faz_prompt_miniverso(lista_final,pokemon):
  vida=total_vida(pokemon)
  print("NAME:",lista_final[1][0].upper(),"|","EFFECTIVE DAMAGE:","|",(round(lista_final[0][0]*0.85,0)/vida)*100,"%","-",(lista_final[0][0]/vida)*100,"%","|", "ACCURACY:",lista_final[3],"|","DAMAGE TYPE:",lista_final[2][0].upper())
  print("NAME:",lista_final[1][1].upper(),"|","EFFECTIVE DAMAGE:","|",(round(lista_final[0][1]*0.85,0)/vida)*100,"%","-",(lista_final[0][1]/vida)*100,"%","|", "ACCURACY:","100","|","DAMAGE TYPE:",lista_final[2][1].upper())

In [ ]:
def chama_COMBATE2(pokemon_a,pokemon_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e,vida_a,vida_e):
  maior_atq_a=atq_a["power"]
  maior_atq_e=atq_e["power"]
  stab_a=faz_stab(atq_a,pokemon_a)
  stab_e=faz_stab(atq_e,pokemon_e)
  resistencia_multi_a=faz_resistencia(pokemon_a,atq_e,dicio_res)
  resistencia_multi_e=faz_resistencia(pokemon_e,atq_a,dicio_res)
  vitoria= COMBATE2(atq_a,atq_e,stab_a,stab_e,resistencia_multi_a,resistencia_multi_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,maior_atq_a,maior_atq_e,velocidade_a,velocidade_e,vida_a,vida_e)
  return vitoria

In [ ]:
def chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e):
  maior_vit=-1
  for id_a in pokemon_a:
    print("Testando:",pokemon_a[id_a]["Nome"],"vs",poke_atual_e["Nome"])
    atq_a=faz_relogio(pokemon_atributos_a[str(id_a)],pokemon_atributos_e[str(pega_id(poke_atual_e))],pokemon_a[id_a],poke_atual_e)
    atq_e=faz_relogio(pokemon_atributos_e[str(pega_id(poke_atual_e))],pokemon_atributos_a[str(id_a)],poke_atual_e,pokemon_a[id_a])
    lista_OD=faz_status_OD(atq_a,pokemon_a[id_a],poke_atual_e)
    status_ofensivo_a=total_stats(lista_OD[0])
    status_defensivo_e=total_stats(lista_OD[1])
    lista_OD=faz_status_OD(atq_e,poke_atual_e,pokemon_a[id_a])
    status_ofensivo_e=total_stats(lista_OD[0])
    status_defensivo_a=total_stats(lista_OD[1])
    vida_e=vida_time_e[str(pega_id(poke_atual_e))]
    vitoria=chama_COMBATE2(pokemon_a[id_a],poke_atual_e,atq_e,atq_a,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e,vida_time_a[id_a],vida_e)
    if vitoria/TESTE>maior_vit:
      maior_vit=vitoria/TESTE
      poke_atual_a=pokemon_a[id_a]
  return poke_atual_a


In [ ]:
def chama_atq(atq,pokemon_a,pokemon_e):
  lista_OD=faz_status_OD(atq,pokemon_a,pokemon_e)
  status_ofensivo=total_stats(lista_OD[0])
  status_defensivo=total_stats(lista_OD[1])
  stab=faz_stab(atq,pokemon_a)
  resistencia_multi=faz_resistencia(poke_atual_e,atq,dicio_res)
  dano=faz_atq2(atq,atq["power"],status_ofensivo,status_defensivo,stab,resistencia_multi)
  return dano



# CARREGA API

In [ ]:
poke_dex={}
cont=1
pokemon_dic={}
pokemon_dic2={}
dicio_res={}
while cont<=ALCANCE:
  # print(cont)
  pokemon_dic[cont]=requesta_a_festa(URL+BUSCA+str(cont))
  pokemon_dic2[cont]=requesta_a_festa(URL+BUSCA2+str(cont))
  poke_dex[cont]=faz_dex(pokemon_dic,pokemon_dic2,cont)
  cont+=1
dicio_res=cria_dicio_res()
#grava_json(poke_dex,"pokedex.json")


# Demonstração ataque

In [ ]:
pokemon_a=cria_poke(ALIADO)
pokemon_e=cria_poke(INIMIGO)
id_a=pega_id(pokemon_a)
id_e=pega_id(pokemon_e)
atq=cria_atq(pokemon_a)

In [ ]:
lista_OD=faz_status_OD(atq,pokemon_a,pokemon_e)
status_ofensivo=total_stats(lista_OD[0])
status_defensivo=total_stats(lista_OD[1])
stab=faz_stab(atq,pokemon_a)
resistencia_multi=faz_resistencia(pokemon_e,atq,dicio_res)

In [ ]:
vida_e=total_vida(pokemon_e)
dano=faz_atq2(atq,atq["power"],status_ofensivo,status_defensivo,stab,resistencia_multi)
print(pokemon_a["Nome"],"usou:"+atq["name"].upper())
print((dano/vida_e)*100,"% de dano!")

# Demonstração universo

In [ ]:
pokemon_a=cria_poke(ALIADO)
pokemon_e=cria_poke(INIMIGO)
id_a=pega_id(pokemon_a)
id_e=pega_id(pokemon_e)

In [ ]:
# A ATACA
lista_final=mini_universo_combate(pokemon_a,pokemon_e,id_a)
print(pokemon_a["Nome"])
faz_prompt_miniverso(lista_final,pokemon_e)

In [ ]:
# E ATACA
lista_final=mini_universo_combate(pokemon_e,pokemon_a,id_e)
print(pokemon_e["Nome"])
faz_prompt_miniverso(lista_final,pokemon_a)

# Demonstração cria e analisa

In [ ]:
#RHYDOM VS RHYDOM
pokemon_a=poke_dex[112]
pokemon_atributos_a={}
id_a=112
pokemon_atributos_a[112]={"name":"RHYDON","IVs":{"Vida":31,"Ataque":31,"Defesa":31,"Ataque especial":31,"Defesa especial":31,"Velocidade":31},
                                    "EVs":{"Vida":252,"Ataque":252,"Defesa":4,"Ataque especial":0,"Defesa especial":0,"Velocidade":0},
                                     "atq":{1:{"name":"earthquake","type":{"name":"ground"},"damage_class":{"name":"physical"},"power":100,"accuracy":100},2:{"name":"stone-edge","type":{"name":"rock"},"damage_class":{"name":"physical"},"power":100,"accuracy":80},3:{"name":"brick-break","type":{"name":"fighting"},"damage_class":{"name":"physical"},"power":75,"accuracy":100},4:{"name":"poison-jab","type":{"name":"poison"},"damage_class":{"name":"physical"},"power":80,"accuracy":100}}}

pokemon_e=poke_dex[112]
id_e=112
pokemon_atributos_e={}
pokemon_atributos_e[112]={"name":"RHYDON","IVs":{"Vida":31,"Ataque":31,"Defesa":31,"Ataque especial":31,"Defesa especial":31,"Velocidade":31},
                                    "EVs":{"Vida":252,"Ataque":252,"Defesa":4,"Ataque especial":0,"Defesa especial":0,"Velocidade":0},
                                     "atq":{1:{"name":"earthquake","type":{"name":"ground"},"damage_class":{"name":"physical"},"power":100,"accuracy":100},2:{"name":"stone-edge","type":{"name":"rock"},"damage_class":{"name":"physical"},"power":100,"accuracy":80},3:{"name":"brick-break","type":{"name":"fighting"},"damage_class":{"name":"physical"},"power":75,"accuracy":100},4:{"name":"poison-jab","type":{"name":"poison"},"damage_class":{"name":"physical"},"power":80,"accuracy":100}}}                                    

In [ ]:
#MEW ID=A
pokemon_a=poke_dex[151]
pokemon_atributos_a={}
id_a=151
pokemon_atributos_a[151]={"name":"MEW","IVs":{"Vida":31,"Ataque":31,"Defesa":31,"Ataque especial":31,"Defesa especial":31,"Velocidade":31},
                                    "EVs":{"Vida":252,"Ataque":0,"Defesa":0,"Ataque especial":252,"Defesa especial":4,"Velocidade":0},
                                     "atq":{1:{"name":"shadow-ball","type":{"name":"ghost"},"damage_class":{"name":"special"},"power":80,"accuracy":100},2:{"name":"psychic","type":{"name":"psychic"},"damage_class":{"name":"special"},"power":90,"accuracy":100},3:{"name":"thunderbolt","type":{"name":"electric"},"damage_class":{"name":"special"},"power":90,"accuracy":100},4:{"name":"ice-beam","type":{"name":"ice"},"damage_class":{"name":"special"},"power":90,"accuracy":100}}}



# pokemon_atributos_a[151]={"name":"MEW","IVs":{"Vida":31,"Ataque":31,"Defesa":31,"Ataque especial":31,"Defesa especial":31,"Velocidade":31},
#                                     "EVs":{"Vida":252,"Ataque":0,"Defesa":0,"Ataque especial":252,"Defesa especial":4,"Velocidade":0},
#                                      "atq":{1:{"name":"flame-thrower","type":{"name":"fire"},"damage_class":{"name":"special"},"power":90,"accuracy":100},2:{"name":"psychic","type":{"name":"psychic"},"damage_class":{"name":"special"},"power":90,"accuracy":100},3:{"name":"thunderbolt","type":{"name":"electric"},"damage_class":{"name":"special"},"power":90,"accuracy":100},4:{"name":"ice-beam","type":{"name":"ice"},"damage_class":{"name":"special"},"power":90,"accuracy":100}}}
                                 


In [ ]:
#MEWTWO ID=E
pokemon_e=poke_dex[150]
pokemon_atributos_e={}
id_e=150
pokemon_atributos_e[150]={"name":"MEWTWO","IVs":{"Vida":31,"Ataque":31,"Defesa":31,"Ataque especial":31,"Defesa especial":31,"Velocidade":31},
                                    "EVs":{"Vida":4,"Ataque":0,"Defesa":0,"Ataque especial":252,"Defesa especial":0,"Velocidade":252},
                                     "atq":{1:{"name":"psychic","type":{"name":"psychic"},"damage_class":{"name":"special"},"power":90,"accuracy":100},2:{"name":"ice-beam","type":{"name":"ice"},"damage_class":{"name":"special"},"power":90,"accuracy":100},3:{"name":"fire-blast","type":{"name":"fire"},"damage_class":{"name":"special"},"power":110,"accuracy":85},4:{"name":"aura-sphere","type":{"name":"fighting"},"damage_class":{"name":"special"},"power":80,"accuracy":100}}}


CRIA DINÂMICAMENTE

In [ ]:
#CRIA A
pokemon_atributos_a=cria_pra_batalha(pokemon_dic,ALIADO)
id_a=pega_id(pokemon_atributos_a)
pokemon_a=poke_dex[id_a]


In [ ]:
#CRIA E
pokemon_atributos_e=cria_pra_batalha(pokemon_dic,INIMIGO)
id_e=pega_id(pokemon_atributos_e)
pokemon_e=poke_dex[id_e]

# CRIA E ANALISA

In [ ]:
atq_a=faz_relogio(pokemon_atributos_a[id_a],pokemon_atributos_e[id_e],pokemon_a,pokemon_e)
atq_e=faz_relogio(pokemon_atributos_e[id_e],pokemon_atributos_a[id_a],pokemon_e,pokemon_a)
velocidade_a=total_stats(pokemon_a["atributos"]["Velocidade"],pokemon_atributos_a[id_a]["EVs"]["Velocidade"],pokemon_atributos_a[id_a]["IVs"]["Velocidade"])
velocidade_e=total_stats(pokemon_e["atributos"]["Velocidade"],pokemon_atributos_e[id_e]["EVs"]["Velocidade"],pokemon_atributos_e[id_e]["IVs"]["Velocidade"])

In [ ]:
#A:ATK E:DEF
lista_OD=faz_status_OD(atq_a,pokemon_a,pokemon_e)
if(atq_a["damage_class"]["name"]=="special"):
  EV_a=pokemon_atributos_a[id_a]["EVs"]["Ataque especial"]
  IV_a=pokemon_atributos_a[id_a]["IVs"]["Ataque especial"]
  EV_e=pokemon_atributos_e[id_e]["EVs"]["Defesa especial"]
  IV_e=pokemon_atributos_e[id_e]["IVs"]["Defesa especial"]
else:
  EV_a=pokemon_atributos_a[id_a]["EVs"]["Ataque"]
  IV_a=pokemon_atributos_a[id_a]["IVs"]["Ataque"]
  EV_e=pokemon_atributos_e[id_e]["EVs"]["Defesa"]
  IV_e=pokemon_atributos_e[id_e]["IVs"]["Defesa"]
status_ofensivo_a=total_stats(lista_OD[0],EV_a,IV_a)
status_defensivo_e=total_stats(lista_OD[1],EV_e,IV_e)


In [ ]:
#E:ATK A:DEF
lista_OD=faz_status_OD(atq_e,pokemon_e,pokemon_a)
if(atq_e["damage_class"]["name"]=="special"):
  EV_e=pokemon_atributos_e[id_e]["EVs"]["Ataque especial"]
  IV_e=pokemon_atributos_e[id_e]["IVs"]["Ataque especial"]
  EV_a=pokemon_atributos_a[id_a]["EVs"]["Defesa especial"]
  IV_a=pokemon_atributos_a[id_a]["IVs"]["Defesa especial"]
else:
  EV_e=pokemon_atributos_e[id_e]["EVs"]["Ataque"]
  IV_e=pokemon_atributos_e[id_e]["IVs"]["Ataque"]
  EV_a=pokemon_atributos_a[id_a]["EVs"]["Defesa"]
  IV_a=pokemon_atributos_a[id_a]["IVs"]["Defesa"]
status_ofensivo_e=total_stats(lista_OD[0],EV_e,IV_e)
status_defensivo_a=total_stats(lista_OD[1],EV_a,IV_a)


In [ ]:
vitoria=chama_COMBATE(pokemon_a,pokemon_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e)

In [ ]:
chance=vitoria/TESTE
if(chance==0):
  print("Nenhuma chance de sucesso! Troque imediatamente!")
elif(chance<0.40):
  print("chance de sucesso:",str(chance*100)+"%. Troque imediatamente!")
elif(chance<0.60):
  print("chance de sucesso:",str(chance*100)+"%. Considere fortemente trocar!")
elif(chance<0.70):
  print("chance de sucesso:",str(chance*100)+"%. Trocar é uma consideração!")
elif(chance<1.0):
  print("chance de sucesso:",str(chance*100)+"%. Fica!")
elif(chance==1.0):
  print("A vitoria é certa!")

In [ ]:
print(atq_a["name"])
print(atq_e["name"])

# UNIVERSO VS UNIVERSO

In [ ]:
pokemon_a=cria_poke(ALIADO)
id_a=pega_id(pokemon_a)

In [ ]:
pokemon_e=cria_poke(INIMIGO)
id_e=pega_id(pokemon_e)

In [ ]:
atq_a=universo_move(id_a,pokemon_a,pokemon_e)

In [ ]:
atq_e=universo_move(id_e,pokemon_e,pokemon_a)

In [ ]:
#A:ATK E:DEF
velocidade_a=total_stats(pokemon_a["atributos"]["Velocidade"])
lista_OD=faz_status_OD(atq_a,pokemon_a,pokemon_e)
status_ofensivo_a=total_stats(lista_OD[0])
status_defensivo_e=total_stats(lista_OD[1])

In [ ]:
#E:ATK A:DEF
velocidade_e=total_stats(pokemon_e["atributos"]["Velocidade"])
lista_OD=faz_status_OD(atq_e,pokemon_e,pokemon_a)
status_ofensivo_e=total_stats(lista_OD[0])
status_defensivo_a=total_stats(lista_OD[1])

In [ ]:
vitoria=chama_COMBATE(pokemon_a,pokemon_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e)

In [ ]:
chance=vitoria/TESTE
if(chance==0):
  print("Nenhuma chance de sucesso! Troque imediatamente!")
elif(chance<0.40):
  print("chance de sucesso:",str(chance*100)+"%. Troque imediatamente!")
elif(chance<0.60):
  print("chance de sucesso:",str(chance*100)+"%. Considere fortemente trocar!")
elif(chance<0.70):
  print("chance de sucesso:",str(chance*100)+"%. Trocar é uma consideração!")
elif(chance<1.0):
  print("chance de sucesso:",str(chance*100)+"%. Fique!")
elif(chance==1.0):
  print("A vitoria é certa!")
# print(atq_a["name"])
# print(atq_e["name"])

# MULTIVERSO

In [ ]:
pokemon_e=cria_poke(INIMIGO)
id_e=pega_id(pokemon_e)
pokemon_a=abre_json("time.json")
pokemon_atributos_a=abre_json("atqs.json")
maior_vit=-1

In [ ]:
for id_a in pokemon_a:
  atq_a=faz_relogio(pokemon_atributos_a[id_a],pokemon_atributos_a[id_a],pokemon_a[id_a],pokemon_e)
  atq_e=universo_move(id_e,pokemon_e,pokemon_a[id_a])
  lista_OD=faz_status_OD(atq_a,pokemon_a[id_a],pokemon_e)
  status_ofensivo_a=total_stats(lista_OD[0])
  status_defensivo_e=total_stats(lista_OD[1])
  lista_OD=faz_status_OD(atq_e,pokemon_e,pokemon_a[id_a])
  status_ofensivo_e=total_stats(lista_OD[0])
  status_defensivo_a=total_stats(lista_OD[1])
  vitoria=chama_COMBATE(pokemon_a[id_a],pokemon_e,atq_a,atq_e,status_ofensivo_a,status_ofensivo_e,status_defensivo_a,status_defensivo_e,velocidade_a,velocidade_e)
  if vitoria/TESTE>maior_vit:
    maior_vit=vitoria/TESTE
    maior_poke=pokemon_a[id_a]
    maior_atq=atq_a["name"]

In [ ]:
maior_atq=maior_atq.replace("-"," ").upper()
print(maior_poke["Nome"])
print(maior_atq)
print("Chances de vitoria:",maior_vit*100,"%")

# RED VS BLUE

In [ ]:
pokemon_a=abre_json("time.json")
pokemon_e=abre_json("time2.json")
pokemon_atributos_a=abre_json("atqs.json")
pokemon_atributos_e=abre_json("atqs2.json")

In [ ]:
vida_time_a={}
vida_time_e={}
for t in pokemon_a:
  vida_time_a[t]=total_vida(pokemon_a[t])
for t in pokemon_e:
  vida_time_e[t]=total_vida(pokemon_e[t])
poke_atual_a=pokemon_a[escolhe_aleatorio(pokemon_a)]
poke_atual_e=pokemon_e[escolhe_aleatorio(pokemon_e)]
cont=0

In [ ]:
print(poke_atual_a["Nome"],"!")
print(poke_atual_e["Nome"],"!")
while len(vida_time_a)>0 and len(vida_time_e)>0:
  cont+=1
  print("TURNO",cont,"!")
  velocidade_a=total_stats(poke_atual_a["atributos"]["Velocidade"])
  velocidade_e=total_stats(poke_atual_e["atributos"]["Velocidade"])
  troca_a=troca_ou_fica(pokemon_a,pokemon_atributos_a,pokemon_atributos_e,poke_atual_a,poke_atual_e,vida_time_a,vida_time_e[str(pega_id(poke_atual_e))])
  troca_e=troca_ou_fica(pokemon_e,pokemon_atributos_e,pokemon_atributos_a,poke_atual_e,poke_atual_a,vida_time_e,vida_time_a[str(pega_id(poke_atual_a))])
  if(velocidade_a>velocidade_e):
    if(troca_a[0]==True):
      print("Sai",poke_atual_a["Nome"],"-A",)
      poke_atual_a=troca_a[1]
      print("Entra",poke_atual_a["Nome"],"-A",)
    if(troca_e[0]==True):
      print("Sai",poke_atual_e["Nome"],"-E",)
      poke_atual_e=troca_e[1]
      print("Entra",poke_atual_e["Nome"],"-E",)
    if(troca_a[0]==False):
      dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
      vida_time_e[str(pega_id(poke_atual_e))]-=dano
      print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
      if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
        vida_time_e.pop(str(pega_id(poke_atual_e)))
        pokemon_e.pop(str(pega_id(poke_atual_e)))
        pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
        if(len(vida_time_e)==0):
          break
        poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
        print("Entra",poke_atual_e["Nome"],"-E")
        continue
    if(troca_e[0]==False):
      dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
      vida_time_a[str(pega_id(poke_atual_a))]-=dano
      print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
      if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
        vida_time_a.pop(str(pega_id(poke_atual_a)))
        pokemon_a.pop(str(pega_id(poke_atual_a)))
        pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
        if(len(vida_time_a)==0):
          break
        poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
        print("Entra",poke_atual_a["Nome"],"-A")
        continue
  elif(velocidade_e>velocidade_a):
    if(troca_e[0]==True):
      print("Sai",poke_atual_e["Nome"],"-E",)
      poke_atual_e=troca_e[1]
      print("Entra",poke_atual_e["Nome"],"-E",)
    if(troca_a[0]==True):
      print("Sai",poke_atual_a["Nome"],"-A",)
      poke_atual_a=troca_a[1]
      print("Entra",poke_atual_a["Nome"],"-A",)
    if(troca_e[0]==False):
      dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
      vida_time_a[str(pega_id(poke_atual_a))]-=dano
      print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
      if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
        vida_time_a.pop(str(pega_id(poke_atual_a)))
        pokemon_a.pop(str(pega_id(poke_atual_a)))
        pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
        if(len(vida_time_a)==0):
          break
        poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
        print("Entra",poke_atual_a["Nome"],"-A")
        continue
    if(troca_a[0]==False):
      dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
      vida_time_e[str(pega_id(poke_atual_e))]-=dano
      print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
      if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
        vida_time_e.pop(str(pega_id(poke_atual_e)))
        pokemon_e.pop(str(pega_id(poke_atual_e)))
        pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
        if(len(vida_time_e)==0):
          break
        poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
        print("Entra",poke_atual_e["Nome"],"-E")
        continue
  elif(velocidade_e==velocidade_a):
    empate=random.randrange(1,3)
    if(empate==1):
      if(troca_a[0]==True):
        print("Sai",poke_atual_a["Nome"],"-A",)
        poke_atual_a=troca_a[1]
        print("Entra",poke_atual_a["Nome"],"-A",)
      if(troca_e[0]==True):
        print("Sai",poke_atual_e["Nome"],"-E",)
        poke_atual_e=troca_e[1]
        print("Entra",poke_atual_e["Nome"],"-E",)
    if(troca_a[0]==False):
      dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
      vida_time_e[str(pega_id(poke_atual_e))]-=dano
      print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
      if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
        vida_time_e.pop(str(pega_id(poke_atual_e)))
        pokemon_e.pop(str(pega_id(poke_atual_e)))
        pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
        if(len(vida_time_e)==0):
            break
        poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
        print("Entra",poke_atual_e["Nome"],"-E")
        continue
    if(troca_e[0]==False):
      dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
      vida_time_a[str(pega_id(poke_atual_a))]-=dano
      print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
      if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
        vida_time_a.pop(str(pega_id(poke_atual_a)))
        pokemon_a.pop(str(pega_id(poke_atual_a)))
        pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
        if(len(vida_time_a)==0):
          break
        poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
        print("Entra",poke_atual_a["Nome"],"-A")
        continue
    else:
      if(troca_e[0]==True):
        print("Sai",poke_atual_e["Nome"],"-E",)
        poke_atual_e=troca_e[1]
        print("Entra",poke_atual_e["Nome"],"-E",)
      if(troca_a[0]==True):
        print("Sai",poke_atual_a["Nome"],"-A",)
        poke_atual_a=troca_a[1]
        print("Entra",poke_atual_a["Nome"],"-A",)
    if(troca_e[0]==False):
      dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
      vida_time_a[str(pega_id(poke_atual_a))]-=dano
      print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
      if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
        vida_time_a.pop(str(pega_id(poke_atual_a)))
        pokemon_a.pop(str(pega_id(poke_atual_a)))
        pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
        if(len(vida_time_a)==0):
          break
        poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
        print("Entra",poke_atual_a["Nome"],"-A")
        continue
    if(troca_a[0]==False):
      dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
      vida_time_a[str(pega_id(poke_atual_a))]-=dano
      print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"E->A")
      if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
        vida_time_e.pop(str(pega_id(poke_atual_e)))
        pokemon_e.pop(str(pega_id(poke_atual_e)))
        pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
        if(len(vida_time_e)==0):
          break
        poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
        print("Entra",poke_atual_e["Nome"],"-E")
        continue

In [ ]:
if(len(pokemon_a)>len(pokemon_e)):
  print("Red venceu")
else:
  print("Blue venceu")

In [ ]:
vitoria_a=0
vitoria_e=0
for tempo in range(5):
  pokemon_a=abre_json("time.json")
  pokemon_e=abre_json("time2.json")
  pokemon_atributos_a=abre_json("atqs.json")
  pokemon_atributos_e=abre_json("atqs2.json")
  vida_time_a={}
  for t in pokemon_a:
    vida_time_a[t]=total_vida(pokemon_a[t])
  vida_time_e={}
  for t in pokemon_e:
    vida_time_e[t]=total_vida(pokemon_e[t])
  poke_atual_a=pokemon_a[escolhe_aleatorio(pokemon_a)]
  poke_atual_e=pokemon_e[escolhe_aleatorio(pokemon_e)]
  cont=0
  print(poke_atual_a["Nome"],"!")
  print(poke_atual_e["Nome"],"!")
  while len(vida_time_a)>0 and len(vida_time_e)>0:
    cont+=1
    print("TURNO",cont,"!")
    velocidade_a=total_stats(poke_atual_a["atributos"]["Velocidade"])
    velocidade_e=total_stats(poke_atual_e["atributos"]["Velocidade"])
    troca_a=troca_ou_fica(pokemon_a,pokemon_atributos_a,pokemon_atributos_e,poke_atual_a,poke_atual_e,vida_time_a,vida_time_e[str(pega_id(poke_atual_e))])
    troca_e=troca_ou_fica(pokemon_e,pokemon_atributos_e,pokemon_atributos_a,poke_atual_e,poke_atual_a,vida_time_e,vida_time_a[str(pega_id(poke_atual_a))])
    if(velocidade_a>velocidade_e):
      if(troca_a[0]==True):
        print("Sai",poke_atual_a["Nome"],"-A",)
        poke_atual_a=troca_a[1]
        print("Entra",poke_atual_a["Nome"],"-A",)
      if(troca_e[0]==True):
        print("Sai",poke_atual_e["Nome"],"-E",)
        poke_atual_e=troca_e[1]
        print("Entra",poke_atual_e["Nome"],"-E",)
      if(troca_a[0]==False):
        dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
        vida_time_e[str(pega_id(poke_atual_e))]-=dano
        print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
        if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
          vida_time_e.pop(str(pega_id(poke_atual_e)))
          pokemon_e.pop(str(pega_id(poke_atual_e)))
          pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
          if(len(vida_time_e)==0):
            break
          poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
          print("Entra",poke_atual_e["Nome"],"-E")
          continue
      if(troca_e[0]==False):
        dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
        vida_time_a[str(pega_id(poke_atual_a))]-=dano
        print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
        if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
          vida_time_a.pop(str(pega_id(poke_atual_a)))
          pokemon_a.pop(str(pega_id(poke_atual_a)))
          pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
          if(len(vida_time_a)==0):
            break
          poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
          print("Entra",poke_atual_a["Nome"],"-A")
          continue
    elif(velocidade_e>velocidade_a):
      if(troca_e[0]==True):
        print("Sai",poke_atual_e["Nome"],"-E",)
        poke_atual_e=troca_e[1]
        print("Entra",poke_atual_e["Nome"],"-E",)
      if(troca_a[0]==True):
        print("Sai",poke_atual_a["Nome"],"-A",)
        poke_atual_a=troca_a[1]
        print("Entra",poke_atual_a["Nome"],"-A",)
      if(troca_e[0]==False):
        dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
        vida_time_a[str(pega_id(poke_atual_a))]-=dano
        print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
        if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
          vida_time_a.pop(str(pega_id(poke_atual_a)))
          pokemon_a.pop(str(pega_id(poke_atual_a)))
          pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
          if(len(vida_time_a)==0):
            break
          poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
          print("Entra",poke_atual_a["Nome"],"-A")
          continue
      if(troca_a[0]==False):
        dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
        vida_time_e[str(pega_id(poke_atual_e))]-=dano
        print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
        if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
          vida_time_e.pop(str(pega_id(poke_atual_e)))
          pokemon_e.pop(str(pega_id(poke_atual_e)))
          pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
          if(len(vida_time_e)==0):
            break
          poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
          print("Entra",poke_atual_e["Nome"],"-E")
          continue
    elif(velocidade_e==velocidade_a):
      empate=random.randrange(1,3)
      if(empate==1):
        if(troca_a[0]==True):
          print("Sai",poke_atual_a["Nome"],"-A",)
          poke_atual_a=troca_a[1]
          print("Entra",poke_atual_a["Nome"],"-A",)
        if(troca_e[0]==True):
          print("Sai",poke_atual_e["Nome"],"-E",)
          poke_atual_e=troca_e[1]
          print("Entra",poke_atual_e["Nome"],"-E",)
      if(troca_a[0]==False):
        dano=chama_atq(troca_a[2],poke_atual_a,poke_atual_e)
        vida_time_e[str(pega_id(poke_atual_e))]-=dano
        print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"A->E")
        if(vida_time_e[str(pega_id(poke_atual_e))]<=0):
          vida_time_e.pop(str(pega_id(poke_atual_e)))
          pokemon_e.pop(str(pega_id(poke_atual_e)))
          pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
          if(len(vida_time_e)==0):
              break
          poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
          print("Entra",poke_atual_e["Nome"],"-E")
          continue
      if(troca_e[0]==False):
        dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
        vida_time_a[str(pega_id(poke_atual_a))]-=dano
        print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
        if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
          vida_time_a.pop(str(pega_id(poke_atual_a)))
          pokemon_a.pop(str(pega_id(poke_atual_a)))
          pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
          if(len(vida_time_a)==0):
            break
          poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
          print("Entra",poke_atual_a["Nome"],"-A")
          continue
      else:
        if(troca_e[0]==True):
          print("Sai",poke_atual_e["Nome"],"-E",)
          poke_atual_e=troca_e[1]
          print("Entra",poke_atual_e["Nome"],"-E",)
        if(troca_a[0]==True):
          print("Sai",poke_atual_a["Nome"],"-A",)
          poke_atual_a=troca_a[1]
          print("Entra",poke_atual_a["Nome"],"-A",)
      if(troca_e[0]==False):
        dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
        vida_time_a[str(pega_id(poke_atual_a))]-=dano
        print(poke_atual_e["Nome"],"usa",troca_e[2]["name"],"em",poke_atual_a["Nome"],"E->A")
        if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
          vida_time_a.pop(str(pega_id(poke_atual_a)))
          pokemon_a.pop(str(pega_id(poke_atual_a)))
          pokemon_atributos_a.pop(str(pega_id(poke_atual_a)))
          if(len(vida_time_a)==0):
            break
          poke_atual_a=chama_chama_combate2(pokemon_atributos_a,pokemon_atributos_e,pokemon_a,poke_atual_e,vida_time_a,vida_time_e)
          print("Entra",poke_atual_a["Nome"],"-A")
          continue
      if(troca_a[0]==False):
        dano=chama_atq(troca_e[2],poke_atual_e,poke_atual_a)
        vida_time_a[str(pega_id(poke_atual_a))]-=dano
        print(poke_atual_a["Nome"],"usa",troca_a[2]["name"],"em",poke_atual_e["Nome"],"E->A")
        if(vida_time_a[str(pega_id(poke_atual_a))]<=0):
          vida_time_e.pop(str(pega_id(poke_atual_e)))
          pokemon_e.pop(str(pega_id(poke_atual_e)))
          pokemon_atributos_e.pop(str(pega_id(poke_atual_e)))
          if(len(vida_time_e)==0):
            break
          poke_atual_e=chama_chama_combate2(pokemon_atributos_e,pokemon_atributos_a,pokemon_e,poke_atual_a,vida_time_e,vida_time_a)
          print("Entra",poke_atual_e["Nome"],"-E")
          continue
  if(len(pokemon_a)>len(pokemon_e)):
    vitoria_a+=1
  else:
    vitoria_e+=1

In [ ]:
if vitoria_a>vitoria_e:
  print("Red venceu")
else:
  print("Blue Venceu")

In [ ]:
#MATRIZ COM TAXA DE SUCESSO PARA VER QUEM É O MELHOR PARA ABRIR